In [5]:


import os, pandas as pd
# Add asset from file system
states = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/statelatlong.csv')
states.head()



,state,latitude,longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [12]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

def distance( lat1, lon1, lat2, lon2 ):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance;

#print("Result:", distance(52.2296756, 21.0122287, 52.406374, 16.9251681))
#print("Should be:", 278.546, "km")

In [13]:
data = []
for s1 in states['state']:
    lat1 = states.latitude[states.state == s1]
    lat1 = states.longitude[states.state == s1]
    for s2 in states['state']:
        lat2 = states.latitude[states.state == s2]
        lat2 = states.longitude[states.state == s2]
        dist = distance(lat1, lon1, lat2, lon2)
        data.append([s1, s2, dist])
        
distances = pd.DataFrame(data=data, columns=['from', 'to', 'value'])
distances.head()

,from,to,value
0,Alabama,Alabama,0.444132
1,Alabama,Alaska,7298.154924
2,Alabama,Arizona,2740.712970
3,Alabama,Arkansas,620.884033
4,Alabama,California,3658.397237


In [14]:
distances.to_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/distance.csv', index=False)